## DIA 094: Agregar Filtro por Rango de Fechas a la Gráfica Personalizada

Hoy aprenderás a permitir que el usuario elija un rango de fechas y vea únicamente las predicciones que hizo dentro de ese intervalo.
Esto le da más control sobre los datos y mejora la experiencia de análisis.

✅ ¿Qué aprenderás hoy?

📌 Filtrar registros en Flask usando fechas.

📌 Recoger fechas desde un formulario HTML.

📌 Actualizar la gráfica en base al filtro aplicado.

🧩 1. Ruta Nueva: /mi-grafica-fechas-filtrada
Agrega esto en tu app.py:

python
Copiar
Editar
from datetime import datetime

@app.route('/mi-grafica-fechas-filtrada', methods=['GET', 'POST'])
@login_required
def mi_grafica_fechas_filtrada():
    labels, valores = [], []

    if request.method == 'POST':
        fecha_inicio = request.form['fecha_inicio']
        fecha_fin = request.form['fecha_fin']

        if fecha_inicio and fecha_fin:
            # Convertir a datetime
            inicio = datetime.strptime(fecha_inicio, '%Y-%m-%d')
            fin = datetime.strptime(fecha_fin, '%Y-%m-%d')

            registros = Registro.query.filter(
                Registro.user_id == current_user.id,
                Registro.fecha >= inicio,
                Registro.fecha <= fin
            ).all()

            fechas = [r.fecha.date() for r in registros]
            conteo = Counter(fechas)

            labels = [f.strftime('%Y-%m-%d') for f in sorted(conteo.keys())]
            valores = [conteo[f] for f in sorted(conteo.keys())]

    return render_template('mi_grafica_fechas_filtrada.html', labels=labels, valores=valores)
🖼️ 2. Template mi_grafica_fechas_filtrada.html
html
Copiar
Editar
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <title>Gráfica Filtrada</title>
    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
</head>
<body class="bg-light">
<div class="container mt-5">
    <h2 class="text-center">📅 Gráfica por Fecha (Filtrada)</h2>
    <form method="POST" class="row g-3 mb-4">
        <div class="col-md-5">
            <label for="fecha_inicio" class="form-label">Desde:</label>
            <input type="date" class="form-control" name="fecha_inicio" required>
        </div>
        <div class="col-md-5">
            <label for="fecha_fin" class="form-label">Hasta:</label>
            <input type="date" class="form-control" name="fecha_fin" required>
        </div>
        <div class="col-md-2 align-self-end">
            <button type="submit" class="btn btn-primary">Filtrar</button>
        </div>
    </form>

    <canvas id="graficoFiltrado" width="400" height="200"></canvas>
</div>

<script>
const ctx = document.getElementById('graficoFiltrado').getContext('2d');
const grafico = new Chart(ctx, {
    type: 'bar',
    data: {
        labels: {{ labels|tojson }},
        datasets: [{
            label: 'Predicciones por Día',
            data: {{ valores|tojson }},
            backgroundColor: 'rgba(255, 205, 86, 0.6)',
            borderColor: 'rgba(255, 205, 86, 1)',
            borderWidth: 1
        }]
    },
    options: {
        scales: {
            y: { beginAtZero: true },
            x: { title: { display: true, text: 'Fecha' } }
        }
    }
});
</script>
</body>
</html>
🔗 3. Enlace desde otra vista (opcional)
html
Copiar
Editar
<a href="/mi-grafica-fechas-filtrada" class="btn btn-outline-warning">📅 Filtrar mi gráfica</a>
🔍 Líneas Clave Explicadas
datetime.strptime(fecha_inicio, '%Y-%m-%d'): convierte la fecha del formulario a objeto datetime.

Registro.query.filter(...).all(): selecciona solo registros del usuario entre dos fechas.

Chart.js: actualiza con los valores filtrados.

